In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE126133"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE126133"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE126133.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE126133.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE126133.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Distinct fibroblast functional states drive clinical outcomes in ovarian cancer and are regulated by TCF21"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['subject id: 65846', 'subject id: 67794', 'subject id: 68584', 'subject id: 70535', 'subject id: 70924', 'subject id: 71029', 'subject id: 71377', 'subject id: 71423', 'subject id: 71853', 'subject id: 72130', 'subject id: 72143', 'subject id: 72199'], 1: ['tissue: high-grade serous ovarian cancer (HGSOC)'], 2: ['cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133-'], 3: ['cell type: CAF', 'cell type: epithelial']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os

# First, analyze the metadata and determine if this dataset is suitable
# Create the clinical_data DataFrame with the sample characteristics
clinical_data = pd.DataFrame({0: ['subject id: 65846', 'subject id: 67794', 'subject id: 68584', 'subject id: 70535', 'subject id: 70924', 'subject id: 71029', 'subject id: 71377', 'subject id: 71423', 'subject id: 71853', 'subject id: 72130', 'subject id: 72143', 'subject id: 72199'], 
                              1: ['tissue: high-grade serous ovarian cancer (HGSOC)'] * 12, 
                              2: ['cell marker: CD31-CD45-EpCAM-CD49e+', 'cell marker: CD31-CD45-EpCAM+CD133+', 'cell marker: CD31-CD45-EpCAM+CD133-'] * 4, 
                              3: ['cell type: CAF', 'cell type: epithelial'] * 6})

# 1. Gene Expression Data Availability
# Based on the background and the presence of cell markers, this appears to be a gene expression dataset
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: We can infer this from the cell type (row 3)
trait_row = 3  # Using cell type as it differentiates between CAF and epithelial cells

# Age and gender are not specified in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cell type to binary: 1 for CAF, 0 for epithelial"""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "caf":
        return 1
    elif value.lower() == "epithelial":
        return 0
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion - not used in this dataset"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion - not used in this dataset"""
    return None

# 3. Save Metadata - initial filtering
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                              cohort=cohort, 
                              info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Transpose the clinical data to match the expected format
    # The geo_select_clinical_features function expects samples as columns
    clinical_data_t = clinical_data.transpose()
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data_t,
        trait="Cell_Type", 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [1.0], 1: [0.0], 2: [1.0], 3: [0.0], 4: [1.0], 5: [0.0], 6: [1.0], 7: [0.0], 8: [1.0], 9: [0.0], 10: [1.0], 11: [0.0]}
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE126133.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs (ILMN_xxxxxxx format), not human gene symbols
# They need to be mapped to gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the preview, we can see:
# - 'ID' column contains the probe identifiers (ILMN_xxxxxxx format)
# - 'Symbol' column contains the gene symbols we need

# 2. Get a gene mapping dataframe using the appropriate columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Number of probe-to-gene mappings: {len(mapping_df)}")
print("First few mappings:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"After mapping: {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
print("First few genes:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and aggregate duplicate gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Number of probe-to-gene mappings: 44837
First few mappings:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
After mapping: 21464 genes and 34 samples
First few genes:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


After normalization: 20259 genes and 34 samples


Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE126133.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data has already been normalized in the previous step
# No need to normalize again, we can directly use the gene_data from the previous step

# 2. Load the clinical data from the file since we need it for linking
# Loading clinical data from file to ensure it's available
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)

# Diagnostic information
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Gene data shape: {gene_data.shape}")
print("Clinical data preview:")
print(clinical_data.head())

# Ensure proper orientation of the datasets
# Clinical data should have traits as rows and samples as columns
clinical_data_t = clinical_data.T if clinical_data.shape[0] > clinical_data.shape[1] else clinical_data
# Gene data should have genes as rows and samples as columns
gene_data_t = gene_data.T if gene_data.shape[0] < gene_data.shape[1] else gene_data

print(f"Transposed clinical data shape: {clinical_data_t.shape}")
print(f"Transposed gene data shape: {gene_data_t.shape}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_t, gene_data_t)
print(f"Linked data initial shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Cell_Type")  # Using the correct trait name from step 2
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Cell_Type")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from CAF and epithelial cells in ovarian cancer samples."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Clinical data shape: (1, 12)
Gene data shape: (20259, 34)
Clinical data preview:
             0    1    2    3    4    5    6    7    8    9   10   11
Cell_Type  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0
Transposed clinical data shape: (1, 12)
Transposed gene data shape: (20259, 34)
Linked data initial shape: (46, 20260)
Linked data preview (first 5 rows, 5 columns):
   Cell_Type  A1BG  A1BG-AS1  A1CF  A2M
0        1.0   NaN       NaN   NaN  NaN
1        0.0   NaN       NaN   NaN  NaN
2        1.0   NaN       NaN   NaN  NaN
3        0.0   NaN       NaN   NaN  NaN
4        1.0   NaN       NaN   NaN  NaN
After handling missing values, linked data shape: (0, 1)
Quartiles for 'Cell_Type':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Cell_Type' in this dataset is fine.

Abnormality detected in the cohort: GSE126133. Preprocessing failed.
Data quality check failed. Linked data not saved.
